In [1]:
# Import needed libraries 
import pandas as pd 
import os
from datetime import timedelta
import numpy as np 
from scipy.signal import argrelextrema
import alpaca_trade_api as tradeapi 
import matplotlib.pyplot as plt 
import matplotlib.dates as mpdates
from mplfinance.original_flavor import candlestick_ohlc
from dotenv import load_dotenv

In [2]:
#Load .env environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
APCA_API_KEY_ID = os.getenv("APCA_API_KEY")
APCA_API_SECRET_KEY = os.getenv("APCA_SECRET_KEY")
ALPACA_ENDPOINT_KEY = os.getenv("ALPACA_END_POINT")

#ensuring api keys are correct 
#print(os.getenv("APCA_API_KEY_ID"))
#print(os.getenv("APCA_API_SECRET_KEY"))
#print(os.getenv("ALPACA_ENDPOINT_KEY"))

# Create the Alpaca API object
api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, api_version="v2",base_url= ALPACA_ENDPOINT_KEY)

In [4]:
# Set the ticker
ticker = "QQQ"

# Set timeframe to "1 Minute"
timeframe = "1Min"

# Set start and end datetimes of 3 years from Today
start_date = pd.Timestamp("2021-01-21", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-01-21", tz="America/New_York").isoformat()

# Get 3 years worth of historical data for TSLA
ticker_data = api.get_bars(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
).df

display(ticker_data.head())
display(ticker_data.tail())

sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/QQQ/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/QQQ/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/QQQ/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/QQQ/bars 3 more time(s)...


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-01-21 09:00:00+00:00,325.28,325.28,325.28,9,325.28,402,325.306219
2021-01-21 09:01:00+00:00,325.50,325.53,325.46,15,325.46,1326,325.493997
2021-01-21 09:02:00+00:00,325.52,325.60,325.51,35,325.54,4958,325.535841
2021-01-21 09:03:00+00:00,325.60,325.60,325.51,8,325.51,1139,325.584267
2021-01-21 09:04:00+00:00,325.51,325.55,325.51,4,325.55,219,325.529726


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2023-01-20 22:50:00+00:00,282.7200,282.7200,282.7200,16,282.7200,497,282.721831
2023-01-20 22:54:00+00:00,282.7000,282.7000,282.7000,13,282.7000,678,282.700147
2023-01-20 22:56:00+00:00,282.7500,282.7500,282.7500,8,282.7500,247,282.730243
2023-01-20 22:58:00+00:00,282.7499,282.7499,282.7499,17,282.7499,604,282.747997
2023-01-20 22:59:00+00:00,282.7500,282.7500,282.7500,71,282.7500,1398,282.747697


In [14]:
#dropping trade_count and vwap column
updated_ticker_data = ticker_data.drop(columns=['trade_count', 'vwap'])

#checking results
updated_ticker_data

,close,high,low,open,volume
18648.375000,325.2800,325.2800,325.2800,325.2800,402
18648.375694,325.5000,325.5300,325.4600,325.4600,1326
18648.376389,325.5200,325.6000,325.5100,325.5400,4958
18648.377083,325.6000,325.6000,325.5100,325.5100,1139
18648.377778,325.5100,325.5500,325.5100,325.5500,219
...,...,...,...,...,...
19377.951389,282.7200,282.7200,282.7200,282.7200,497
19377.954167,282.7000,282.7000,282.7000,282.7000,678
19377.955556,282.7500,282.7500,282.7500,282.7500,247
19377.956944,282.7499,282.7499,282.7499,282.7499,604


In [ ]:
# Function to find local maximas and minimas
def find_local_maximas_minimas(ohlc, window_range, smooth=False, smoothing_period=10):
    local_max_arr = []
    local_min_arr = []

    if smooth:
        smooth_close = ohlc["close"].rolling(window=smoothing_period).mean().dropna()
        local_max = argrelextrema(smooth_close.values, np.greater)[0]
        local_min = argrelextrema(smooth_close.values, np.less)[0]   
    else:
        local_max = argrelextrema(ohlc["close"].values, np.greater)[0]
        local_min = argrelextrema(ohlc["close"].values, np.less)[0]   

    for i in local_max:
        if (i>window_range) and (i<len(ohlc)-window_range):
            local_max_arr.append(ohlc.iloc[i-window_range:i+window_range]['close'].idxmax())
   
    for i in local_min:
        if (i>window_range) and (i<len(ohlc)-window_range):
            local_min_arr.append(ohlc.iloc[i-window_range:i+window_range]['close'].idxmin())  

    maxima  = pd.DataFrame(ohlc.loc[local_max_arr])
    minima  = pd.DataFrame(ohlc.loc[local_min_arr])
    max_min = pd.concat([maxima, minima]).sort_index()
    max_min = max_min[~max_min.index.duplicated()]

    return max_min


In [ ]:
# Function to find double tops and bottoms patterns
def find_doubles_patterns(max_min):
    patterns_tops = []
    patterns_bottoms = []

    for i in range(5, len(max_min)):  
        window = max_min.iloc[i-5:i]
        
        if window.index[-1] - window.index[0] > 50:      
            continue   
            
        a, b, c, d, e = window.iloc[0:5,1]
        if a<b and a<d and c<b and c<d and e<b and e<d and b>d:
            patterns_tops.append((window.index[0], window.index[-1]))

        if a>b and a>d and c>b and c>d and e>b and e>d and b<d:
            patterns_bottoms.append((window.index[0], window.index[-1]))
            
return patterns_tops, patterns_bottoms

In [ ]:
# Function to save the plots of the analysis
def save_plots(ohlc, patterns, max_min, filename):
    for i, pattern in enumerate(patterns):
        fig, ax  = plt.subplots(figsize=(15,7)) 
        start_, end_ = pattern
        idx = max_min.loc[start_-100:end_+100].index.values.tolist()
        ohlc_copy = ohlc.copy()
        ohlc_copy.loc[:,"Index"] = ohlc_copy.index

        max_min_idx = max_min.loc[start_:end_].index.tolist()

        candlestick_ohlc(ax, ohlc_copy.loc[idx, ["Index","Open", "High", "Low", "Close"] ].values, width=0.1, colorup='green', colordown='red', alpha=0.8)   
        ax.plot(max_min_idx, max_min.loc[start_:end_].values[:,1], color='orange')

        ax.grid(True)
        ax.set_xlabel('Index')
        ax.set_ylabel('Price')

        plt_dir = os.path.join(os.getcwd(), "images", "analysis")
        if not os.path.exists(plt_dir):
            os.makedirs(plt_dir)

        fn = f"{filename}-{i}.png"
        file = os.path.join(plt_dir, fn)
        plt.savefig(file, format="png")

        print(f"Completed {round((i+1)/len(patterns),2)*100}%")

    return 

if __name__ == "__main__":
    # Find all the local minimas and maximas
    window_range = 10  # Local range for finding min and max
    max_min = find_local_maximas_minimas(ticker_data, window_range, smooth=True)

    # Find the tops and bottoms
    patterns_tops, patterns_bottoms = find_doubles_patterns(max_min)

    # Plots for Double Tops and Bottoms
    save_plots(ticker_data, patterns_tops, max_min, f"{ticker}-double-tops")
    save_plots(ticker_data, patterns_bottoms, max_min, f"{ticker}-double-bottoms")
